# Cracking AES-ECB

In [7]:
import secrets

# Generate a secret key
secret_key = secrets.token_bytes(16)
cracked_message = b"Hello, world! This is a secret message; in theory, we shouldn't be able to decrypt it without the key."

In [8]:
from Crypto.Cipher import AES


def aes_ecb_encrypt(plaintext: bytes, key: bytes):
    # Pad the plaintext to a multiple of 16 bytes
    plaintext += b"\x00" * (16 - len(plaintext) % 16)
    # Encrypt the plaintext with AES-128-ECB
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.encrypt(plaintext)


def aes_ecb_decrypt(ciphertext: bytes, key: bytes):
    # Decrypt the ciphertext with AES-128-ECB
    cipher = AES.new(key, AES.MODE_ECB)
    return cipher.decrypt(ciphertext)


def post_to_server(message: bytes):
    # Append the secret message to the user's message before encrypting
    ciphertext = aes_ecb_encrypt(message + cracked_message, secret_key)
    # Send the ciphertext back to the user
    return ciphertext

In [9]:
# Let's try to post a message to the server
post_to_server(b'')

b'\xd3Z\xd3 \xdf\x1f~\xfc\xb1\xd4P\xb0pb\x89:\xfd\xeaxU7f}\xed\x96Y\xb3\x7f\xf7\'\x88\x12lp\xd2\x9c\x8cf\x9d\xdf\x04\x90\xf4\xd2\xf9h\x94P\xc1."\xd5D\x1a\x01\xab\x970+\xb4G\x97|\xf0\x85p\xfe[\xaf\xa2\n1\xd9\x0e\xa9\x00~\xc2w\x1b\xf7h\xe7\x02\x94\x9d\xbbYy\xb7\xf4\xcdY\x85\xedQ\xab\\\xd7\xad28-\xfc\xff\xf8ayH\x80\xb3\x99'

In [10]:
# Inject 15 bytes of known plaintext
known_plaintext = b'\x00' * 15
ciphertext = post_to_server(known_plaintext)
first_block = ciphertext[:16]
print(f"First block: {first_block.hex()}")

# Brute-force the last byte of the secret message
for i in range(256):
    guessed_byte = bytes((i,))
    guessed_plaintext = known_plaintext + bytes([i])
    ciphertext = post_to_server(guessed_plaintext)
    # Check if the first block of the ciphertext matches the first block of the secret message
    if ciphertext[:16] == first_block:
        print(f"Guessed byte: {guessed_byte.decode()}")
        break

First block: 445619079bf203c32b2e33d4c1ec4376
Guessed byte: H


In [11]:
# Let's generalize the attack to recover the entire secret message!
def crack_ecb_secret_message():
    # Initialize the secret message to an empty byte string
    cracked_message = b""
    # Iterate over the length of the secret message
    for i in range(len(post_to_server(b""))):
        # Inject 15 bytes of known plaintext
        known_plaintext = b"\x00" * (15 - (i % 16))
        # Find the ciphertext block containing the next byte of the secret message
        block_index = i // 16 * 16
        block_slice = slice(block_index, block_index + 16)
        block = post_to_server(known_plaintext)[block_slice]
        # Brute-force the next byte of the secret message
        for j in range(256):
            guessed_byte = bytes((j,))
            guessed_plaintext = known_plaintext + cracked_message + guessed_byte
            ciphertext = post_to_server(guessed_plaintext)
            # Check if the chosen block of the ciphertext matches the chosen block of the secret message
            if ciphertext[block_slice] == block:
                cracked_message += guessed_byte
                break
    return cracked_message


print(f"Secret message: {crack_ecb_secret_message().decode()}")

Secret message: Hello, world! This is a secret message; in theory, we shouldn't be able to decrypt it without the key.          
